<a href="https://colab.research.google.com/github/amriT2044524/geemap_amrit/blob/main/spectral_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import matplotlib.pyplot as plt

# Import the wget module
import ee
import geemap
from skimage.transform import resize

In [4]:
ee.Authenticate() # https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication
ee.Initialize(project='ee-athapa2')

In [5]:
# 1. Define the point of interest (Long, Lat)
point = ee.Geometry.Point(85, 28)

# 2. Filter Sentinel-2 Image Collection (Example: cloud-free median composite for 2024)
s2_collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(point)
    .filterDate('2024-01-01', '2024-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) # Less than 10% cloud cover
    .median() # Create a median composite image
)

# 3. Select the bands for the spectral profile (Surface Reflectance bands)
# Bands are scaled by 10000 (Source 1.1)
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
s2_image = s2_collection.select(bands)

In [6]:
# 4. Create an interactive map centered on the point
m = geemap.Map(center=[28, 85], zoom=10)

# Add the Sentinel-2 image (using a False Color Composite)
vis_params = {'min': 0, 'max': 3000, 'bands': ['B8', 'B4', 'B3']}
m.add_layer(s2_image, vis_params, 'Sentinel-2 Image')

# Add the point as a marker
m.add_marker([28, 85], tooltips='Point (85, 28)')

# Add the interactive plotting tool (allows clicking anywhere to plot)
# This uses the timeseries inspector or plot_gui functionality of geemap
m.add_plot_gui()

# Display the map in Colab
m